In [2]:
import os
import unicodedata

import torch
import pandas as pd
from tqdm import tqdm
import fitz  # PyMuPDF
import pymupdf4llm


from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig
)
from accelerate import Accelerator

# Langchain 관련
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
# from llama_index.core import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

from langchain.retrievers import EnsembleRetriever, ContextualCompressionRetriever
from langchain_community.retrievers import BM25Retriever
# from langchain_teddynote.retrievers import KiwiBM25Retriever

from langchain.retrievers.document_compressors import FlashrankRerank


# psm pdf 함수 관련
import glob
import pdfplumber
from langchain_community.document_loaders import PyPDFLoader


/home/psm/anaconda3/envs/anz/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
MARKDOWN_SEPARATORS = [
    "\n#{1,6} ",
    "```\n",
    "\n\\*\\*\\*+\n",
    "\n---+\n",
    "\n___+\n",
    "\n\n",
    "\n",
    " ",
    "",
]

In [3]:
from kiwipiepy import Kiwi

kiwi = Kiwi()

def kiwi_tokenize(text):
    return [token.form for token in kiwi.tokenize(text)]

In [4]:
def process_pdfs_psm(file_path, chunk_size=512, chunk_overlap=32):
    pdf_files = glob.glob(file_path)
    pages = []
    
    # PDF 파일에서 페이지 정보를 로드합니다.
    for pdf_file_path in pdf_files:
        loader = PyPDFLoader(pdf_file_path)
        pages.extend(loader.load_and_split())

    extracted_tables = {}

    # 각 PDF 파일에 대해 테이블 추출 작업을 수행합니다.
    for pdf_file_path in pdf_files:
        with pdfplumber.open(pdf_file_path) as pdf:
            for i, page in enumerate(pdf.pages):
                # 이미지 처리 로직 추가 가능

                # 테이블 추출
                table = page.extract_table()
                if table:
                    df = pd.DataFrame(table[1:], columns=table[0])
                    markdown_table = df.to_markdown()
                    extracted_tables[(pdf_file_path, i)] = markdown_table
                else:
                    extracted_tables[(pdf_file_path, i)] = "No table found"

    all_content = ""
    # 추출된 테이블을 페이지 컨텐츠에 추가하고 전체 텍스트를 하나로 합칩니다.
    for page in pages:
        pdf_file = page.metadata['source']
        page_number = page.metadata['page']
        
        if (pdf_file, page_number) in extracted_tables:
            table_content = extracted_tables[(pdf_file, page_number)]
            page_content = page.page_content
            updated_content = f"{page_content}\n\nTable extracted from page {page_number}:\n{table_content}"
            page.page_content = updated_content
        
        all_content += "\n\n" + page.page_content  # 모든 페이지 컨텐츠를 하나의 문자열로 연결

    # 전체 텍스트를 chunk로 분할
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        add_start_index=True,
        strip_whitespace=True,
        separators=["\n\n", "\n"]  # MARKDOWN_SEPARATORS로 설정해야 하는 구체적인 값을 설정
    )
    
    chunks = splitter.split_text(all_content)
    return [Document(page_content=t) for t in chunks]


In [4]:
def process_pdf_by_markdown(file_path, chunk_size=512, chunk_overlap=32):
    md_text = pymupdf4llm.to_markdown(file_path)
    
    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
    ]

    md_header_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, strip_headers=False)
    md_chunks = md_header_splitter.split_text(md_text)

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap
    )

    splits = text_splitter.split_documents(md_chunks)
    return splits

def process_pdf(file_path, chunk_size=512, chunk_overlap=32):
    """PDF 텍스트 추출 후 chunk 단위로 나누기"""
    # PDF 파일 열기
    doc = fitz.open(file_path)
    text = ''
    # 모든 페이지의 텍스트 추출
    for page in doc:
        text += page.get_text()
    # 텍스트를 chunk로 분할
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        add_start_index=True,
        strip_whitespace=True,
        separators=MARKDOWN_SEPARATORS
    )
    chunk_temp = splitter.split_text(text)
    # Document 객체 리스트 생성
    chunks = [Document(page_content=t) for t in chunk_temp]
    return chunks




In [7]:



def create_vector_db(chunks, model_path="intfloat/multilingual-e5-large"):
    """FAISS DB 생성"""
    # 임베딩 모델 설정
    model_kwargs = {'device': 'cuda'}
    encode_kwargs = {'normalize_embeddings': True}
    embeddings = HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    # FAISS DB 생성 및 반환
    db = FAISS.from_documents(chunks, embedding=embeddings)
    
    return db

def normalize_path(path):
    """경로 유니코드 정규화"""
    return unicodedata.normalize('NFC', path)


def process_pdfs_from_dataframe(df, base_directory):
    """딕셔너리에 pdf명을 키로해서 DB, retriever 저장"""
    pdf_databases = {}
    unique_paths = df['Source_path'].unique()

    for path in tqdm(unique_paths, desc="Processing PDFs"):
        # 경로 정규화 및 절대 경로 생성
        normalized_path = normalize_path(path)
        full_path = os.path.normpath(os.path.join(base_directory, normalized_path.lstrip('./'))) if not os.path.isabs(normalized_path) else normalized_path

        pdf_title = os.path.splitext(os.path.basename(full_path))[0]
        print(f"Processing {pdf_title}...")

        # PDF 처리 및 벡터 DB 생성
        chunks = process_pdf(full_path)
        
        # # Markdown 형식으로 처리해서 벡터 DB 생성
        chunks = process_pdf_by_markdown(full_path)
        # chunks 생성 후 검증
        
        # chunks = process_pdfs_psm(full_path)
        if not chunks:
            raise ValueError("No chunks generated from the PDFs. Check if the PDFs contain readable text or tables.")

        db = create_vector_db(chunks)
        
        # Retriever 생성
        retriever = db.as_retriever(search_type="mmr",
                                    search_kwargs={'k': 5, 'fetch_k': 10})
        
        bm25_retriever = BM25Retriever.from_documents(chunks, preprocess_func=kiwi_tokenize,
                                                      search_kwargs={'k': 5, 'fetch_k': 10})

        # initialize the ensemble retriever
        ensemble_retriever = EnsembleRetriever(
            retrievers=[bm25_retriever, retriever], weights=[0.5, 0.5],
        )

        # 결과 저장
        pdf_databases[pdf_title] = {
                'db': db,
                'retriever': ensemble_retriever
        }
    return pdf_databases

def documents_from_pdfs(df, base_directory):
    """딕셔너리에 pdf명을 키로해서 DB, retriever 저장"""
    pdf_databases = {}
    unique_paths = df['Source_path'].unique()
    documents = []
    
    for path in tqdm(unique_paths, desc="Processing PDFs"):
        # 경로 정규화 및 절대 경로 생성
        normalized_path = normalize_path(path)
        full_path = os.path.normpath(os.path.join(base_directory, normalized_path.lstrip('./'))) if not os.path.isabs(normalized_path) else normalized_path

        pdf_title = os.path.splitext(os.path.basename(full_path))[0]
        print(f"Processing {pdf_title}...")

        # PDF 처리 및 벡터 DB 생성
        chunks = process_pdfs_psm(full_path)
        documents.extend(chunks)
    return documents




In [8]:
base_directory = '/home/psm/model/data' # Your Base Directory
df = pd.read_csv('/home/psm/model/data/test.csv')
# documents = documents_from_pdfs(df, base_directory)
pdf_databases = process_pdfs_from_dataframe(df, base_directory)

Processing PDFs:   0%|          | 0/9 [00:00<?, ?it/s]

Processing 중소벤처기업부_혁신창업사업화자금(융자)...


/home/psm/anaconda3/envs/anz/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
Processing PDFs:  11%|█         | 1/9 [00:08<01:04,  8.02s/it]

Processing 보건복지부_부모급여(영아수당) 지원...


Processing PDFs:  22%|██▏       | 2/9 [00:12<00:42,  6.10s/it]

...

Processing PDFs:  78%|███████▊  | 7/9 [00:44<00:13,  6.85s/it]

Processing 「FIS 이슈&포커스」 22-2호 《재정성과관리제도》...


Processing PDFs:  89%|████████▉ | 8/9 [00:51<00:06,  6.62s/it]

Processing 「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...


Processing PDFs: 100%|██████████| 9/9 [00:56<00:00,  6.32s/it]


In [9]:
# Helper function for printing docs
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [10]:
def normalize_string(s):
    """유니코드 정규화"""
    return unicodedata.normalize('NFC', s)

def format_docs(docs):
    """검색된 문서들을 하나의 문자열로 포맷팅"""
    context = ""
    for doc in docs:
        context += doc.page_content
        context += '\n'
    return context

In [11]:
q_n = 0
query = df['Question'][q_n]
# answer = df['Answer'][q_n]
source = df['Source'][q_n]

normalized_keys = {normalize_string(k): v for k, v in pdf_databases.items()}
retriever = normalized_keys[normalize_string(source)]['retriever']

print(source)
print(query)
# print(answer)
print()

docs = retriever.invoke(query)
pretty_print_docs(docs)

중소벤처기업부_혁신창업사업화자금(융자)
2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?

Document 1:

## 직접 출자 출연 보조 융자 국고보조율(%) 융자율 (%)
○ ○  
3. 지출계획 총괄표  
(단위: 백만원, %)  
|사업명|2022년 결산|2023년|Col4|2024년|증감 (B-A) (B-A)/A|Col7|
|---|---|---|---|---|---|---|
|||당초(A)|수정|확정(B)||(B-A)/A|
|혁신창업사업화자 금(융자)|2,300,000|2,230,000|2,330,000|2,007,800|△222,200|△9.96|  
2023년 2024년  
2022년 증감
사업명  
결산 당초(A) 수정 확정(B) (B-A) (B-A)/A  
혁신창업사업화자  
2,300,000 2,230,000 2,330,000 2,007,800 △222,200 △9.96  
금(융자)
----------------------------------------------------------------------------------------------------
Document 2:

## 사  업  명  
혁신창업사업화자금(융자) (5152-301)
----------------------------------------------------------------------------------------------------
Document 3:

-  2014. 1 지원대상 업력기준을 7년 미만으로 확대  
-  2015. 1 재창업자금을 재도약지원자금(융자)의 내역사업으로 이관  
-  2019. 1 청년전용창업자금을 혁신창업지원자금에 통합 운영  
개발기술사업화자금을 내역사업으로 통합 및 일자리창출촉진자금 신규 지원  
-  2020. 1 미래기술육성자금, 고성장촉진자금 신규 지원  
-  2022. 1 미래기술육성자금 및 고성장촉진자금 사업 폐지  
-  2023. 1 창업기반지원과 신청 대상이 중복인 일자리창출촉진자금을

In [12]:
def setup_llm_pipeline():
    # 4비트 양자화 설정
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    # 모델 ID
    model_id = "rtzr/ko-gemma-2-9b-it"
    # model_id = "/home/psm/model/resource/notebook/results/checkpoint-500"

    # 토크나이저 로드 및 설정
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.use_default_system_prompt = False

    # 모델 로드 및 양자화 설정 적용
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True
    )

    # HuggingFacePipeline 객체 생성
    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        # temperature=0.2,
        return_full_text=False,
        max_new_tokens=300,
    )

    hf = HuggingFacePipeline(pipeline=text_generation_pipeline)

    return hf, tokenizer

In [13]:
# LLM 파이프라인
llm, tokenizer = setup_llm_pipeline()

Loading checkpoint shards: 100%|██████████| 10/10 [00:10<00:00,  1.03s/it]
/home/psm/anaconda3/envs/anz/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


In [14]:
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.retrievers.document_compressors import CrossEncoderReranker

model = HuggingFaceCrossEncoder(model_name="Dongjin-kr/ko-reranker")
compressor = CrossEncoderReranker(model=model, top_n=5)

In [1]:



# 결과를 저장할 리스트 초기화
results = []

# DataFrame의 각 행에 대해 처리
for _, row in tqdm(df.iterrows(), total=len(df), desc="Answering Questions"):
    # 소스 문자열 정규화
    source = normalize_string(row['Source'])
    question = row['Question']

    # 정규화된 키로 데이터베이스 검색
    normalized_keys = {normalize_string(k): v for k, v in pdf_databases.items()}
    retriever = normalized_keys[source]['retriever']
    
    # Compressed revriever
    compression_retriever = ContextualCompressionRetriever(
        base_compressor=compressor, base_retriever=retriever,
    )

    # RAG 체인 구성
    template = """
    ### 다음 정보를 바탕으로 질문에 답하세요 (간결하게 답변하고 똑같은 단어는 반복하지 마세요):
    {context}

    ### 질문: {question}

    ### 답변:
    """
    prompt = PromptTemplate.from_template(template)
    
    rag_chain = (
        {"context": compression_retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    # 답변 추론
    print(f"Question: {question}")
    full_response = rag_chain.invoke(question)

    print(f"Answer: {full_response}\n")

    # 결과 저장
    results.append({
        "Source": row['Source'],
        "Source_path": row['Source_path'],
        "Question": question,
        "Answer": full_response
    })
    
    


NameError: name 'tqdm' is not defined

In [15]:
new_data = {
    'SAMPLE_ID': ['TRAIN_{:03d}'.format(i) for i in range(len(results))],
    'Answer': ""
}
submit_df = pd.DataFrame(new_data)
submit_df

,SAMPLE_ID,Answer
0,TRAIN_000,
1,TRAIN_001,
2,TRAIN_002,
3,TRAIN_003,
4,TRAIN_004,
...,...,...
95,TRAIN_095,
96,TRAIN_096,
97,TRAIN_097,
98,TRAIN_098,


In [16]:
# 제출용 샘플 파일 로드
submit_df = pd.read_csv("/home/psm/model/data/sample_submission.csv")

# 생성된 답변을 제출 DataFrame에 추가
submit_df['Answer'] = [item['Answer'].split("### 질문")[0].strip() for item in results]
submit_df['Answer'] = submit_df['Answer'].fillna("데이콘")     # 모델에서 빈 값 (NaN) 생성 시 채점에 오류가 날 수 있음 [ 주의 ]


# 결과를 CSV 파일로 저장
submit_df.to_csv("/home/psm/model/data/08166_submission.csv", encoding='UTF-8-sig', index=False)

: 

In [ ]:
# Helper function for printing docs

def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [19]:
# 제출용 샘플 파일 로드
# submit_df = pd.read_csv("/home/psm/model/data/sample_submission.csv")

# 생성된 답변을 제출 DataFrame에 추가
submit_df['Answer'] = [item['Answer'].split("### 질문")[0].strip() for item in results]
submit_df['Answer'] = submit_df['Answer'].fillna("데이콘")     # 모델에서 빈 값 (NaN) 생성 시 채점에 오류가 날 수 있음 [ 주의 ]

# 결과를 CSV 파일로 저장
submit_df.to_csv("/home/psm/model/data/0816_train_infer_3.csv", encoding='UTF-8-sig', index=False)